In [ ]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np

from matplotlib.cm import get_cmap
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.lines import Line2D

In [ ]:
# Lecture des données
data = pd.read_csv('data/flandreau_jobst_internationalcurrencies_data.txt', encoding='cp1252', skiprows=9, header=0, sep='\t')

In [ ]:
# Commençons par créer un fichier attributes en ne gardant que les éléments par pays puis supprimant les lignes qui se répètent
attributes = data[["country_A", "gold", "colony", "debtburden", "rlong", "rshort1900", "rshort1890", "rgdp", "rgdpcap", "poldemo", "coverage"]]
# Sort the dataframe to ensure rows with colony=1 come first
attributes = attributes.sort_values(by=["country_A", "colony"], ascending=[True, False])
# Drop duplicates, keeping the first row (colony=1 will be kept if present)
attributes = attributes.drop_duplicates(subset="country_A", keep="first").reset_index(drop=True)
attributes = attributes.rename(columns={"colony": "is_colonized"})

# Récupérons aussi le nombre de colonies
nb_colonies = data.groupby("country_B")["colony"].sum().reset_index()
nb_colonies = nb_colonies.rename(columns={"colony": "has_colonies"})
attributes = nb_colonies.merge(attributes, left_on="country_B", right_on="country_A", how="left")
attributes = attributes.drop(columns=['country_A'])
attributes = attributes.rename(columns={"country_B": "country"})

In [ ]:
country_mapping = {
    "ARG": "Argentina",
    "AUS": "Australia",
    "AUH": "United Arab Emirates",
    "BEL": "Belgium",
    "BRA": "Brazil",
    "CAN": "Canada",
    "CEY": "Ceylon",
    "CHE": "Switzerland",
    "CHL": "Chile",
    "CHN": "China",
    "COL": "Colombia",
    "CUB": "Cuba",
    "DEU": "Germany",
    "DNK": "Denmark",
    "ECU": "Ecuador",
    "EGY": "Egypt",
    "ESP": "Spain",
    "FIN": "Finland",
    "FRA": "France",
    "GBR": "United Kingdom",
    "GRC": "Greece",
    "HKG": "Hong Kong",
    "ICH": "Sandwich Islands",
    "IND": "India",
    "ITA": "Italy",
    "JAV": "Java",
    "JPN": "Japan",
    "MEX": "Mexico",
    "NLD": "Netherlands",
    "NOR": "Norway",
    "NZL": "New Zealand",
    "OTT": "Ottoman Empire",
    "PER": "Peru",
    "PHL": "Philippines",
    "PRT": "Portugal",
    "PRS": "Persia",
    "ROM": "Romania",
    "RUS": "Russia",
    "SER": "Serbia",
    "SGP": "Singapore",
    "SIA": "Siam",
    "SWE": "Sweden",
    "URY": "Uruguay",
    "USA": "United States",
    "VEN": "Venezuela"}

attributes["country"] = attributes["country"].map(country_mapping)

In [ ]:
attributes.style.hide(axis="index")

# Première approche
***Description de la structure du graph ?***

Il faut d'abord bien préciser la signification des données : un lien allant de $A$ vers $B$ signifie que le pays $B$ possède un marché des changes intégrant la devise du pays $A$. En l'absence de tout autre lien, cela signifie qu'il est nécessaire de passer par le pays $B$ pour échanger la devise du pays $A$, c'est-à-dire que le pays $A$ est dépendant du pays $B$ pour ses échanges extérieurs.

In [ ]:
# Création d'un graph orienté avec la variable 1900 (possible de pondérer le graph pour utiliser les 3 ? Ou de faire avec le commerce ?)
graph_change_1900 = nx.DiGraph()

for _, row in data.iterrows():
    if row["quote1900"] == 1:
        graph_change_1900.add_edge(row["country_A"], row["country_B"])

In [ ]:
n = graph_change_1900.number_of_nodes()
L = graph_change_1900.number_of_edges()
density = L/(n*(n-1))
print(f"Nombre de sommets : {n}")
print(f"Nombre d'arêtes : {L}")
print(f"Densité : {density:.4f}")

On constate d'emblée que le réseau est très peu dense, avec seulement 218 liens sur les $45*44=1980$ possibles, ce qui correspond à une densité de 0.11 : ceci indique que la majorité des pays ne sont pas directement connectés sur le marché des changes, autrement dit, que celui-ci fonctionne de manière centralisée, ou que de nombreux pays ne participent pas aux échanges internationaux.

In [ ]:
isolates = list(nx.isolates(graph_change_1900))
print(f"Proportion de pays isolés : {len(isolates) / n:.0f}")

no_incoming = [node for node in graph_change_1900.nodes() if graph_change_1900.in_degree(node) == 0]
no_incoming_share = len(no_incoming) / n if n > 0 else 0
print(f"Proportion de pays  sans lien entrant : {100*no_incoming_share:.1f} %")

bidir_edges = sum(1 for u, v in graph_change_1900.edges() if graph_change_1900.has_edge(v, u))
total_edges = graph_change_1900.number_of_edges()
unidir_edges = total_edges - bidir_edges
country_links = unidir_edges + (bidir_edges / 2)
bidir_share = (bidir_edges / 2) / country_links
print(f"Proportion de pays sans liens réciproques : {100*(1-bidir_share):.1f} %")

print(f"La réciprocité moyenne vaut {nx.reciprocity(graph_change_1900):.3f}")

On ne trouve aucun pays isolé, ce qui écarte l'hypothèse de pays ne participant pas aux échanges internationaux. En revanche, 44,4 % des pays ne sont destinataires d'aucun lien, c'est-à-dire qu'ils doivent utiliser des marchés étrangers pour échanger leur monnaie nationale.

Si l'on s'intéresse aux liens entre pays sans considérer leur direction, mais seulement leur caractère unilatéral ou réciproque, on constate que 72,5 % des pays n'entretiennent aucun lien réciproque.

In [ ]:
def truncate_colormap(cmap_name, min_val=0.2, max_val=1.0, n=100):
    cmap = plt.get_cmap(cmap_name)
    new_cmap = LinearSegmentedColormap.from_list(
        f"{cmap_name}_trunc", cmap(np.linspace(min_val, max_val, n))
    )
    return new_cmap

trunc_oranges = truncate_colormap("Oranges", 0.2, 1.0)

# Determine edge colors
edge_colors = []
for u, v in graph_change_1900.edges():
    if graph_change_1900.has_edge(v, u):
        edge_colors.append("forestgreen")  # reciprocal
    else:
        edge_colors.append("skyblue")  # unidirectional

# Compute node colors based on total degree (in-degree + out-degree)
degrees = dict(graph_change_1900.degree())
node_colors = [degrees[node] for node in graph_change_1900.nodes()]

# Draw graph
plt.figure(figsize=(16, 8))
pos = nx.spring_layout(graph_change_1900, seed=43)

nodes = nx.draw_networkx_nodes(
    graph_change_1900,
    pos,
    node_size=600,
    node_color=node_colors,
    cmap=trunc_oranges)

edges = nx.draw_networkx_edges(
    graph_change_1900,
    pos,
    edge_color=edge_colors)

labels = nx.draw_networkx_labels(graph_change_1900, pos, font_size=9, font_color="white")

legend_elements = [
    Line2D([0], [0], marker='o', color='w', markerfacecolor='forestgreen', markersize=9, label='Unidirectional'),
    Line2D([0], [0], marker='o', color='w', markerfacecolor='skyblue', markersize=9, label='Bidirectional')
]
plt.legend(handles=legend_elements, loc='upper right')

plt.colorbar(nodes, label="Total Degree")
plt.axis("off")
plt.show()

Une représentation graphique fait apparaître :
- Un marché des changes très intégré entre l'Europe et les Etats-Unis, rassemblant une dizaine de pays (GBR, FRA, DEU, USA notamment)
- Un marché des changes secondaires en Asie, rassemblant une demi-douzaine de pays (HKG, SGP, CHN, IND notamment)
- Un relatif isolement des autres pays, qui entretiennent des liens unilatéraux avec quelques pays appartenants aux groupes précédents

*(On pourrait utiliser le nombre de devises échangées plutôt que le nombre de liens pour la coloration.)*

In [ ]:
countries = ["DEU", "HKG", "EGY"]

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

for i, country_B in enumerate(countries):
    # Get the nodes related to country_B
    neighbors = list(graph_change_1900.neighbors(country_B))
    predecessors = list(graph_change_1900.predecessors(country_B))
    nodes_involved = [country_B] + neighbors + predecessors
    subgraph = graph_change_1900.subgraph(nodes_involved)
    restricted_graph = nx.DiGraph(subgraph)

    edge_colors = []
    for u, v in restricted_graph.edges():
        if restricted_graph.has_edge(v, u):
            edge_colors.append("forestgreen")  # reciprocal
        else:
            edge_colors.append("skyblue")  # unidirectional

    pos = nx.spring_layout(restricted_graph)
    ax = axes[i]
    nx.draw_networkx_nodes(restricted_graph, pos, node_size=600, node_color="chocolate", ax=ax)
    nx.draw_networkx_edges(restricted_graph, pos, edge_color=edge_colors, ax=ax)
    nx.draw_networkx_labels(restricted_graph, pos, font_size=9, font_color="white", ax=ax)
    ax.set_title(f"Graph restricted to {country_B} and its neighbors")

plt.tight_layout()
plt.show()

On peut faire apparaître les marchés intégrés en restreignant le graph aux liens réciproques ou en recherchant les cliques...

In [ ]:
graph_change_1900 = nx.DiGraph()

for _, row in data.iterrows():
    if row["quote1900"] == 1:
        graph_change_1900.add_edge(row["country_A"], row["country_B"])

# Filter for reciprocal edges
reciprocal_edges = [(u, v) for u, v in graph_change_1900.edges() if graph_change_1900.has_edge(v, u)]
# Get nodes involved in reciprocal edges
nodes_with_reciprocal_edges = set(u for u, v in reciprocal_edges) | set(v for u, v in reciprocal_edges)

plt.figure(figsize=(8, 5))
pos = nx.spring_layout(graph_change_1900, seed=42)
graph_filtered = graph_change_1900.subgraph(nodes_with_reciprocal_edges)

nx.draw_networkx_nodes(graph_filtered, pos, node_size=600, node_color="chocolate")
nx.draw_networkx_edges(graph_filtered, pos, edgelist=reciprocal_edges, edge_color="skyblue")
nx.draw_networkx_labels(graph_filtered, pos, font_size=9, font_color="white")

plt.axis("off")
plt.show()

In [ ]:
# On peut essayer de formaliser en recherchant des cliques
undirected_graph = graph_change_1900.to_undirected()
cliques = list(nx.find_cliques(undirected_graph ))
largest_cliques = sorted(cliques, key=len, reverse=True)[:1]
print(f"Plus grande clique : {largest_cliques[0]}")

In [ ]:
out_degree = graph_change_1900.out_degree()
in_degree = graph_change_1900.in_degree()

out_degree_centrality = nx.out_degree_centrality(graph_change_1900)
in_degree_centrality = nx.in_degree_centrality(graph_change_1900)
betweenness_centrality = nx.betweenness_centrality(graph_change_1900)
eigenvector_centrality = nx.eigenvector_centrality(graph_change_1900)

df = pd.DataFrame({
    "Country": [node for node, _ in out_degree],
    "Out-Degree": [degree for _, degree in out_degree],
    "In-Degree": [degree for _, degree in in_degree],
    "Out-Degree Centrality": [out_degree_centrality.get(node, 0) for node, _ in out_degree],
    "In-Degree Centrality": [in_degree_centrality.get(node, 0) for node, _ in in_degree],
    "Betweenness Centrality": [betweenness_centrality.get(node, 0) for node, _ in out_degree],
    "Eigenvector Centrality": [eigenvector_centrality.get(node, 0) for node, _ in out_degree]
})

df["In / Out ratio"] = df["In-Degree Centrality"] / df["Out-Degree Centrality"]
df = df[[*df.columns[:df.columns.get_loc("In-Degree Centrality")+1], "In / Out ratio", *df.columns[df.columns.get_loc("In-Degree Centrality")+1:].drop("In / Out ratio")]]

df_sorted = df.sort_values(by="Betweenness Centrality", ascending=False)
df_sorted.style.hide(axis="index").format({
    "Out-Degree": "{:.0f}",
    "In-Degree": "{:.0f}",
    "Out-Degree Centrality": "{:.3f}",
    "In-Degree Centrality": "{:.3f}",
    "Betweenness Centrality": "{:.3f}",
    "Average Centrality": "{:.3f}",
    "In / Out ratio": "{:.3f}",
    "Eigenvector Centrality": "{:.3f}",
})

In [ ]:
pearson_corr = df['In-Degree'].corr(df['Out-Degree'])
print(f"Corrélation entre les centralités entrantes et sortantes : {pearson_corr:.4f}")

... ou en étudiant la centralité.

La **centralité intermédiaire** fait ressortir quelques marchés constituant des points de passage obligés : GBR et DEU, ainsi dans une moindre mesure que FRA, HKG et DNK.

Le ratio entre les **centralités entrantes et sortantes**, c'est-à-dire entre le nombre de devises échangées localement par un pays, et le nombre de pays échangeant sa propre devise, permet de distinguer des situations plus contrastées.
- Les pays colonisateurs : GBR, FRA et ESP, dont la centralité sortante est très supérieure à la centralité entrante, sans doute parce qu'ils entretiennent des liens unilatéraux avec leurs colonnies et des liens bilatéraux avec les autres pays développés.
- Les pays développés : DEU, IT, USA, etc., dont les centralités entrantes et sortantes sont relativement proches, sans doute parce qu'ils entretiennent surtout des liens bilatéraux entre eux.
- Les pays périphériques / dépendants, qui n'ont aucun lien entrant.

La **centralité vectorielle** n'apporte rien par rapport aux analyses précédentes, elle oppose seulement les pays intégrés aux autres.

In [ ]:
# Jetons un oeil à la transitivité
print(f"La transitivité globale est de {nx.average_clustering(graph_change_1900):.3f}")
print(f"La transitivité moyenne est de {nx.transitivity(graph_change_1900):.3f}")

print('La transitivité locale de chaque pays est : ')
for node, coeff in sorted(nx.clustering(graph_change_1900).items(), key=lambda x: x[1]):
    print(f"{node}: {coeff:.3f}")

La transitivité n'est pas très éclairante ?

# Seconde approche
***Expliquer la structure du graph avec les attributs ?***

Intéger les relations commerciales (avec `bitrade`) et financières (avec `rshort`et `rlong`) ? Par exemple, est-ce que l'absence de liens entre pays reflète la faiblesse des échanges commerciaux, ou des différentiels de taux d'intérêt ?

Pour les colonnies : aucun pays Africain dans la base

In [ ]:
# Ajoutons les attributs
attributesdata = attributes.set_index('country').to_dict('index').items()
graph_change_1900.add_nodes_from(attributesdata)

print(nx.get_node_attributes(graph_change_1900, 'gold'))

In [ ]:
# Création d'un graph des colonies 
graph_colony = nx.DiGraph()

for _, row in data.iterrows():
    if row["colony"] == 1:
        graph_colony.add_edge(row["country_A"], row["country_B"])

print(f"Nombre de sommets : {graph_colony.number_of_nodes()}")
print(f"Nombre d'arêtes : {graph_colony.number_of_edges()}")

nx.draw(graph_colony, with_labels=True, node_size=500, font_size=10)
plt.show()

# Bon par contre ça c'est vraiment un tout petit graph, on a très peu de colonies en fait. Ou plus exactement sans doute, peu de colonies qui ont un marché des changes